In [2]:
import pandas as pd
import numpy as np
import csv
import os

In [28]:
dfGitRev = pd.read_csv("../../ch_git_revision.csv")

In [29]:
dfGitRev['component_id'].loc[:] = None
dfGitRev.head()

/tmp/ipykernel_5142/773015420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGitRev['component_id'].loc[:] = None


,commit,path,dir,component_id
0,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/base/thread_collision_warner.h,src/base/,NaN
1,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/base/thread_collision_warner_unittest.cc,src/base/,NaN
2,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/base/thread_local_storage_posix.cc,src/base/,NaN
3,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/base/thread_local_storage_unittest.cc,src/base/,NaN
4,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/base/thread_local_storage_win.cc,src/base/,NaN


In [30]:
dfComp = pd.read_csv("../../ch_component.csv")
dfComp.tail()

,component_id,component_name
86,193,gpu
87,199,docs
88,236,profile
89,238,reporting
90,239,search


In [31]:
dir_path = "content/chromeos/graph/"
dir_path = dir_path.rstrip('/')
dirs = dir_path.split('/')
dirs = dirs[::-1]

for d in dirs:
    if dfComp['component_name'].str.contains(d).any():
        print(f"{d} -- {dfComp.iloc[0]['component_id']}")
        break;

chromeos -- 215


## Label dfGitRev with component_id

In [32]:
import re

# Define a function to update the component_name and component_id columns of a row in df1 based on a match in df2
def update_component(row, dfComp):
    # Create a dictionary mapping component_name to component_id from dfComp
    component_id_mapping = dfComp.set_index('component_name')['component_id'].to_dict()
    
    dir_path = row['dir']
    # Skip rows where the dir column is not a string or is empty
    if not isinstance(dir_path, str) or not dir_path:
        return row
    
    dir_path = dir_path.rstrip('/') # Remove if there is any '/' at the end of the path string
    dirs = dir_path.split('/') # Take the directories into an array
    dirs = dirs[::-1] # Reverting the array
    for d in dirs:
        for comp_name, comp_id in component_id_mapping.items():
            if re.match(f"^{comp_name}.*", d):
                row['component_id'] = component_id_mapping[comp_name]
                break
        if row['component_id'] > 0:
            break;
    return row

# Apply the update_component function to each row of df1 and update the component_name and component_id columns
dfGitRev = dfGitRev.apply(lambda r: update_component(r, dfComp), axis=1)

In [41]:
# Print the updated DataFrame
dfGitRevLabeled = dfGitRev.dropna(subset=['component_id'])
dfGitRevLabeled.to_csv('../../ch_git_revision_labaled.csv')

In [45]:
dfGitRevLabeled

,commit,path,dir,component_id
12,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/chrome/app/theme/back.png,src/chrome/app/theme/,147.0
38,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/chrome/app/chrome_dll_app_mode_mac.mm,src/chrome/app/,147.0
39,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/chrome/app/chrome_dll_main.cc,src/chrome/app/,147.0
40,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/chrome/app/chrome_dll_resource.h,src/chrome/app/,147.0
41,8ced33be1ee63e589afff6f9229b36fe73cfd0f9,src/chrome/app/chrome_dll_version.rc.version,src/chrome/app/,147.0
...,...,...,...,...
6009688,8e749ebccf72b5e3c12f8565a3421b8261d7db20,src/chrome/browser/ui/views/login_view.h,src/chrome/browser/ui/views/,54.0
6009689,8e749ebccf72b5e3c12f8565a3421b8261d7db20,src/chrome/browser/ui/webui/conflicts_ui.h,src/chrome/browser/ui/webui/,3.0
6009690,8e749ebccf72b5e3c12f8565a3421b8261d7db20,src/chrome/browser/ui/webui/favicon_source.h,src/chrome/browser/ui/webui/,3.0
6009691,8e749ebccf72b5e3c12f8565a3421b8261d7db20,src/chrome/browser/ui/webui/html_dialog_ui.cc,src/chrome/browser/ui/webui/,3.0


In [46]:
dfGitRevLabeled['component_id'].isnull().sum()

0

In [47]:
#import re
#
## Define a function to extract the third last word of a string
#def extract_third_last_word(s):
#    if isinstance(s, str):
#        words = re.findall(r'\b\w+\b', s)
#        if len(words) > 2:
#            return words[-3]
#    return ''
#
## Define a function to update the component_name and component_id columns of a row in df1 based on a match in df2
#def update_component(row):
#    # Skip rows where the dir column is not a string or is empty
#    if not isinstance(row['dir'], str) or not row['dir']:
#        return row
#    # Look for a match in df2 based on the third last word of the directory path in df1
#    match = df2[df2['name'].str.contains(extract_third_last_word(row['dir']))]
#    if not match.empty and pd.isnull(row['component_id']):
#        # If a match is found and the component_id is NaN, update the component_name and component_id columns
#        row['component_name'] = match.iloc[0]['name']
#        row['component_id'] = match.iloc[0]['id']
#    return row
#
## Apply the update_component function to each row of df1 and update the component_name and component_id columns
#df1 = df1.apply(update_component, axis=1)
#
## Print the updated DataFrame
#print(df1)